In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime, timedelta

In [8]:
today_URL = "https://extapps.dec.ny.gov/cfmx/extapps/aqi/aqi_forecast.cfm"
today_r = requests.get(today_URL)

In [3]:
yesterday_URL = "https://extapps.dec.ny.gov/cfmx/extapps/aqi/observed.cfm"
yesterday_r = requests.get(yesterday_URL)

In [9]:
tsoup = BeautifulSoup(today_r.content, 'html.parser')
ttable = tsoup.find('table', {'summary': 'Layout for forecast information.'})
trows = ttable.find_all('tr')

In [5]:
ysoup = BeautifulSoup(yesterday_r.content, 'html.parser')
ytable = ysoup.find('table', {'summary': 'Layout for forecast information.'})
yrows = ytable.find_all('tr')

In [12]:
data = pd.DataFrame(columns=['Date','Region','Maximum AQI','AQI Value','Air Quality'])

data = data.astype({
    'Date': 'string',
    'Region': 'string',       
    'Maximum AQI': 'string',  
    'AQI Value': 'string',
    'Air Quality': 'string',        
})

today_df = pd.DataFrame(data)
yesterday_df = pd.DataFrame(data)

In [13]:
def NYCtoDF(rows_item, dataframe):
    for row in rows_item:
        strong_tag = row.find('strong')
        if strong_tag and "NYC" in strong_tag.get_text():
            cells = row.find_all('td')
            dataframe.loc[0, 'Region'] = cells[0].get_text(strip=True)
            dataframe.loc[0, 'Maximum AQI'] = cells[1].get_text(strip=True)
            dataframe.loc[0, 'Air Quality'] = cells[2].get_text(strip=True)
            return dataframe

In [14]:
AQItoday = NYCtoDF(trows, today_df)
AQItoday.loc[0, 'Date'] = datetime.today().strftime('%Y-%m-%d')
AQItoday['AQI Value'] = AQItoday['Maximum AQI'].str.split('(').str[0].astype(float)

In [12]:
AQIyesterday = NYCtoDF(yrows, yesterday_df)
yesterday = datetime.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
AQIyesterday.loc[0, 'Date'] = yesterday

In [15]:
AQItoday

,Date,Region,Maximum AQI,AQI Value,Air Quality
0,2024-12-15,NYC Metro,55(Fine Particles),55.0,Moderate


In [14]:
AQIyesterday

,Date,Region,Maximum AQI,Air Quality
0,2024-09-18,NYC Metro,46(Fine Particles),Good
